In [2]:
import pandas
print("Hello World")

Hello World


In [11]:
teams_data = pandas.read_csv("kaggle_data/Teams.csv")
reg_season_detail = pandas.read_csv("kaggle_data/RegularSeasonDetailedResults.csv")
seasons_data = pandas.read_csv("kaggle_data/Seasons.csv")
tourney_compact = pandas.read_csv("kaggle_data/TourneyCompactResults.csv")
tourney_detail = pandas.read_csv("kaggle_data/TourneyDetailedResults.csv")
tourney_seeds = pandas.read_csv("kaggle_data/TourneySeeds.csv")
tourney_slots = pandas.read_csv("kaggle_data/TourneySlots.csv")